In [1]:
# enable autoreload
%load_ext autoreload
%autoreload 2

# import standard libraries
import os
import sys
from pathlib import Path
from typing import cast, Any, Dict, List, Optional, Union, Tuple, Callable, Awaitable, Iterable, TypeVar, Generic, Type, ClassVar, Set, NamedTuple


In [2]:
from sdis.model.service import DBService, DBBase
from sdis.model.albums import Album, AlbumKeys, AlbumService, AlbumImport

In [3]:
sdis_content = Path('/sdis-content')
out_content = Path('/data/www/')
txt2img_images = sdis_content / 'txt2img-images'

for p in txt2img_images.iterdir():
    if p.is_dir():
        print(p)

/sdis-content/txt2img-images/2023-03-07
/sdis-content/txt2img-images/2023-03-09
/sdis-content/txt2img-images/2023-03-08


In [4]:
p_2023_03_07 = txt2img_images / '2023-03-07'

In [5]:
album_import = AlbumImport(basepath=sdis_content)

alb1 = album_import.load_from_path(p_2023_03_07)
repr(alb1)

"Album(id='album:txt2img-images/2023-03-07', name='2023-03-07', original_path=PosixPath('/sdis-content/txt2img-images/2023-03-07'), generated_path=None, thumbnail_path=None, created=DateTime(2023, 3, 7, 0, 0, 0, tzinfo=Timezone('America/Vancouver')))"

In [6]:
from redis.client import Redis

redis = Redis(db=1)
redis.flushdb()

True

In [7]:
albkeys = AlbumKeys(basepath=sdis_content)
albkey1 = albkeys.album_key(alb1)

In [8]:
conv = DBBase.make_converter()
albdict1 = conv.unstructure(alb1)
albdict1

{'id': 'album:txt2img-images/2023-03-07',
 'name': '2023-03-07',
 'original_path': '/sdis-content/txt2img-images/2023-03-07',
 'generated_path': None,
 'thumbnail_path': None,
 'created': '2023-03-07T00:00:00-08:00'}

In [9]:

redisjson = redis.json()
redisjson.set(albkey1, '$', conv.unstructure(alb1))

True

In [10]:
albkeys: list[str] = list(redis.scan_iter('album:*')) # type: ignore
albkeys

[b'album:txt2img-images/2023-03-07']

In [11]:
alb1_fromdb_result: list = redisjson.get(albkeys[0], '$') # type: ignore
alb1_fromdb: Album = conv.structure(alb1_fromdb_result[0], Album)
alb1_fromdb

Album(id='album:txt2img-images/2023-03-07', name='2023-03-07', original_path=PosixPath('/sdis-content/txt2img-images/2023-03-07'), generated_path=None, thumbnail_path=None, created=DateTime(2023, 3, 7, 0, 0, 0, tzinfo=Timezone('-08:00')))

In [12]:
from urllib.parse import quote

def encode_path_components(path: Path) -> str:
    # Encode each component, allowing spaces to remain unencoded
    encoded_components = [quote(part, safe=" ") for part in path.parts]
    # Join the components back together
    return "/".join(encoded_components)

alb1.original_path.parts

('/', 'sdis-content', 'txt2img-images', '2023-03-07')

In [13]:
alb1_fromdb == alb1

True

In [23]:
alb_txt2img = album_import.load_from_path(txt2img_images)
alb_txt2img_subpaths = [p for p in txt2img_images.iterdir() if p.is_dir()]
alb_txt2img_subalbums = [album_import.load_from_path(p) for p in alb_txt2img_subpaths]
alb_txt2img_subalbum_keys = [album_import.album_key(a) for a in alb_txt2img_subalbums]
alb_txt2img_subalbum_keys


['album:txt2img-images/2023-03-07',
 'album:txt2img-images/2023-03-09',
 'album:txt2img-images/2023-03-08']

In [25]:
alb_txt2img_subalbums_key = album_import.subalbums_key(alb_txt2img)
redis.sadd(alb_txt2img_subalbums_key, *alb_txt2img_subalbum_keys)

3

In [32]:
members = cast(set[bytes], redis.smembers(alb_txt2img_subalbums_key))